In [9]:
import pandas as pd
from pyprojroot import here

In [10]:
df = pd.read_csv(here("data/csv_xlsx/Augmented.csv"))
print(df.shape)
print(df.columns.tolist())
display(df.head(3))

(2002, 10)
['co_name', 'Address', 'Place', 'Zip', 'Bill No', 'Date', 'Product', 'Quantity', 'Rate', 'Amount']


,co_name,Address,Place,Zip,Bill No,Date,Product,Quantity,Rate,Amount
0,Gamma Industries,845 Willow Rd,New York,10001,BIL001,01/01/2023,Monitor,16,116,1856
1,Zeta Enterprises,831 Spruce Ave,Chicago,60601,BIL032,02/01/2023,Laptop,31,57,1767
2,Beta Solutions,209 Pine Rd,Chicago,60601,BIL060,03/01/2023,Mouse,1,219,219


In [11]:
from langchain_community.utilities import SQLDatabase
from sqlalchemy import create_engine
db_path = str(here("data")) + "/test_sqldb.db"
db_path = f"sqlite:///{db_path}"

engine = create_engine(db_path)
# df.to_sql("titanic", engine, index=False)
df.to_sql("Augmented", engine, index=False)

ValueError: Table 'Augmented' already exists.

In [ ]:
db = SQLDatabase(engine=engine)
print(db.dialect)
print(db.get_usable_table_names())
db.run("SELECT * FROM Augmented WHERE Amount < 219;")

sqlite
['Augmented']


"[('Wise PLC', '085 Kimberly Spur Suite 617', 'Mcdowellfort', '97037', 'BIL342', '03/01/2025', 'Headphones', 1, 191, 191), ('Cook Inc', '8018 Michael Drives Apt. 540', 'Cathyview', '93444', 'BIL920', '03/02/2025', 'Keyboard', 3, 46, 138), ('Quinn, Roman and Clayton', '734 Ashley Squares Apt. 372', 'South Lucas', '13319', 'BIL187', '03/03/2025', 'Headphones', 4, 49, 196), ('Taylor, Valdez and Martinez', '65099 Mason Village', 'Port Lorifurt', '94823', 'BIL810', '01/04/2025', 'Keyboard', 4, 35, 140), ('Walker, Wood and Ramos', '9582 Swanson Ports', 'South Josephburgh', '19139', 'BIL919', '01/04/2025', 'Webcam', 5, 39, 195), ('Warren and Sons', '691 Pamela Stravenue', 'South Kevinchester', '96499', 'BIL839', '03/04/2025', 'Router', 1, 90, 90), ('Gibson, Edwards and Peterson', '08584 Timothy Cape Apt. 131', 'Kellyview', '5049', 'BIL186', '02/05/2025', 'Printer', 2, 71, 142), ('Lawson, Rhodes and Rowe', '7889 Kevin Crossing', 'New James', '60697', 'BIL383', '02/06/2025', 'Laptop', 7, 21, 14

In [ ]:
import os
from dotenv import load_dotenv
import warnings
warnings.filterwarnings("ignore")
print("Environment variables are loaded:", load_dotenv())

Environment variables are loaded: True
test by reading a variable: None


In [ ]:
from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv
import os

load_dotenv()

openai_api_key = os.getenv("OPENAI_API_KEY")

llm = ChatOpenAI(
    openai_api_key=openai_api_key,
    model_name="gpt-3.5-turbo",  
    temperature=0.1
)

/var/folders/d7/2dp_fv5d4cx4885m2qfdndfw0000gn/T/ipykernel_8753/1007662375.py:9: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(


In [12]:
from langchain.chains import create_sql_query_chain

chain = create_sql_query_chain(llm, db)
response = chain.invoke({"question": "what is the maximum Amount"})
print(response)

SELECT MAX("Amount") 
FROM "Augmented"


In [13]:
db.run(response)

'[(20000000000000,)]'

In [14]:
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool

write_query = create_sql_query_chain(llm, db)
execute_query = QuerySQLDataBaseTool(db=db)

chain = write_query | execute_query

chain.invoke({"question": "what is the product purchased by Hamilton Ltd on 01/06/2025"})

/var/folders/d7/2dp_fv5d4cx4885m2qfdndfw0000gn/T/ipykernel_8753/234968396.py:4: LangChainDeprecationWarning: The class `QuerySQLDataBaseTool` was deprecated in LangChain 0.3.12 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-community package and should be used instead. To use it run `pip install -U :class:`~langchain-community` and import as `from :class:`~langchain_community.tools import QuerySQLDatabaseTool``.
  execute_query = QuerySQLDataBaseTool(db=db)


"[('Headphones',)]"

In [15]:
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

answer_prompt = PromptTemplate.from_template(
    """Given the following user question, corresponding SQL query, and SQL result, answer the user question. And the SQL table is a transaction table.

Question: {question}
SQL Query: {query}
SQL Result: {result}
Answer: """
)

answer = answer_prompt | llm | StrOutputParser()
chain = (
    RunnablePassthrough.assign(query=write_query).assign(
        result=itemgetter("query") | execute_query
    )
    | answer
)

chain.invoke({"question": "Which Product has the most transactions for Lambda Labs?"})

'The product with the most transactions for Lambda Labs is Tablet, with a total of 2 transactions.'

In [16]:
from langchain_community.agent_toolkits import create_sql_agent
agent_executor = create_sql_agent(llm, db=db, agent_type="openai-tools", verbose=True)

In [18]:
agent_executor.invoke({"input": "Which Product has the most transactions for Lambda Labs?"})



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


Augmented
Invoking: `sql_db_schema` with `{'table_names': 'Augmented'}`
responded: I will query the schema of the "Augmented" table to see the relevant columns for finding the product with the most transactions for Lambda Labs.


CREATE TABLE "Augmented" (
	co_name TEXT, 
	"Address" TEXT, 
	"Place" TEXT, 
	"Zip" TEXT, 
	"Bill No" TEXT, 
	"Date" TEXT, 
	"Product" TEXT, 
	"Quantity" BIGINT, 
	"Rate" BIGINT, 
	"Amount" BIGINT
)

/*
3 rows from Augmented table:
co_name	Address	Place	Zip	Bill No	Date	Product	Quantity	Rate	Amount
Gamma Industries	845 Willow Rd	New York	10001	BIL001	01/01/2023	Monitor	16	116	1856
Zeta Enterprises	831 Spruce Ave	Chicago	60601	BIL032	02/01/2023	Laptop	31	57	1767
Beta Solutions	209 Pine Rd	Chicago	60601	BIL060	03/01/2023	Mouse	1	219	219
*/
Invoking: `sql_db_query` with `{'query': "SELECT Product, COUNT(*) AS Transaction_Count FROM Augmented WHERE co_name = 'Lambda Labs' GROU

{'input': 'Which Product has the most transactions for Lambda Labs?',
 'output': 'The product with the most transactions for Lambda Labs is "Tablet" with a total of 2 transactions.'}